In [1]:
%load_ext autoreload
%autoreload 2

from feature_extraction import Method
from readers import GetProcessedInputData
from ipynb.fs.full.accuracy_metrics import CalculateAccuracy


feature_extraction_methods = [Method.UPPER_CONTOUR]
transcriptions_as_list, keywords_to_search, train_words, test_words = GetProcessedInputData(feature_extraction_methods)

In [2]:
# TODO: For testing only. Remove the entire cell later

# all_train_words = train_words
# all_test_words = test_words

train_words = train_words[:7]
test_words = test_words[:7]

In [3]:
import numpy as np
from dtaidistance import dtw
from compute_distances import ComputeDistances


distances = ComputeDistances(train_words, test_words)

In [4]:
import pickle   
#Put distances dictionary into a pickle file
#Put the name of you feature
file = open("output/UPPER_CONTOUR.pkl", "wb")
pickle.dump(distances, file)
file.close()

In [5]:
import pickle   
#Retrieve the distance dictionary from a pickle file
#Put the name of you feature
file_to_read = open("output/UPPER_CONTOUR.pkl", "rb")
distances_load = pickle.load(file_to_read)
file_to_read.close()

In [6]:
print(distances_load)

{'270-01-01': {'300-02-01': 2.448712441213993, '300-02-02': 4.324303393506464, '300-02-03': 4.381723821182499, '300-02-04': 4.921819526314599, '300-02-05': 7.718092374559931, '300-02-06': 4.793410304513034, '300-02-07': 2.507044862912476}, '270-01-02': {'300-02-01': 2.474095776213498, '300-02-02': 1.960956675586445, '300-02-03': 2.2919413485596767, '300-02-04': 3.695998576345429, '300-02-05': 2.820506267735343, '300-02-06': 2.346573798468874, '300-02-07': 2.330678950775133}, '270-01-03': {'300-02-01': 2.3556546371895197, '300-02-02': 1.831993059395856, '300-02-03': 1.28443083078318, '300-02-04': 3.9927354450505734, '300-02-05': 2.6341286326191025, '300-02-06': 2.1885836741033278, '300-02-07': 3.711463905922475}, '270-01-04': {'300-02-01': 3.8570960631986373, '300-02-02': 3.806601021508875, '300-02-03': 3.0139946907447728, '300-02-04': 1.106887528161737, '300-02-05': 4.883593269120123, '300-02-06': 4.429288985635686, '300-02-07': 2.052995911346755}, '270-01-05': {'300-02-01': 3.44598278

In [ ]:
from keyword_spotting import FindClosestKnownWord, SpotKeyword


# For each test_word, find the train_word that is closest to it.
for test_word in test_words:
  test_word.closest_train_word, test_word.distance_to_closest_train_word = \
    FindClosestKnownWord(test_word, train_words, distances)
    

# Search keywords and save spotted keywords in the list.
outputData = []
for keyword_to_search in keywords_to_search:
  spotted_test_keywords = SpotKeyword(keyword_to_search, test_words)
  if len(spotted_test_keywords):
    print(keyword_to_search)
    print(spotted_test_keywords)
    spottedKeyword = []
    for keyword in spotted_test_keywords:
      spottedKeyword.append(keyword.id)
      spottedKeyword.append(keyword.distance_to_closest_train_word)
      print(keyword.distance_to_closest_train_word)
      keyword.image.show()
    outputData.append([keyword_to_search, *spottedKeyword])
    input()

In [ ]:
# TODO: write output file kws.csv (slide 24)
import csv
with open("kws.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(outputData)

In [ ]:
# Evaluation Metrics - In progress, not finished yet (There might be errors in the logic)
# convert transcriptions to dict - easier and faster access values later
from ipynb.fs.full.accuracy_metrics import CalculateAccuracy
groundTruth = dict()
for sub in transcriptions_as_list:
    groundTruth[sub[0]] = sub[1]

testWords = dict()
for word in test_words:
    testWords[word.id] = word.transcription

predictions = dict()
for item in outputData:
    predictions[item[0]] = item[1::2]

acc = CalculateAccuracy(keywords_to_search, testWords, predictions, groundTruth)


print(acc.CalculateRecall())
print(acc.CalculatePrecision())
